# Matching Clusters Across Induction and Collection

In [22]:
import numpy as np
import pandas as pd

In [23]:
allclusters = pd.read_pickle("/Users/user/data/research/proton-deuteron/csv/allclusters.pkl") # deuterons from within vertices bounding box + one track protons
print(allclusters.columns); allclusters.shape

Index(['event_idx', 'run', 'subrun', 'event', 'file_path', 'event_index',
       'particle_type', 'plane', 'cluster_idx', 'area', 'max_intensity',
       'min_intensity', 'mean_intensity', 'total_intensity', 'centroid_x',
       'centroid_y', 'bbox_min_row', 'bbox_min_col', 'bbox_max_row',
       'bbox_max_col', 'width', 'height', 'aspect_ratio', 'compactness',
       'image_intensity', 'matrix_transformed', 'column_maxes'],
      dtype='object')


(438575, 27)

In [24]:
# CUTS
print(allclusters.shape)
greaterThanThree = allclusters[allclusters['height'] > 3]; print(greaterThanThree.shape) # Remove all clusters with length = 1, noise
noConstants = greaterThanThree[greaterThanThree['column_maxes'].map(lambda x: len(set(x)) > 1)]; print(noConstants.shape) # Remove all clusters where the max ADC for each wire is the same (no change in energy deposition)

(438575, 27)
(105907, 27)
(105896, 27)


In [25]:
maxes = noConstants['column_maxes'].values

normalised = []

for i, signal in enumerate(maxes):
    
    minimum = 0
    maximum = np.max(signal)

    if maximum > minimum:
        normalised_signal = (signal - minimum) / (maximum - minimum)
    else:
        normalised_signal = np.zeros_like(signal, dtype=float)
        print(f'Error: Max not greater than min in signal {i}.')

    normalised.append(normalised_signal)

noConstants['normalised'] = normalised
adcdiff = [np.diff(signal) for signal in normalised]; noConstants['adcdiff'] = adcdiff
maxdiff = [np.max(diff) for diff in adcdiff]; noConstants['maxdiff'] = maxdiff

/var/folders/cm/m55y72dj6cd5lkwrfdb2bpnr0000gp/T/ipykernel_52880/1492737920.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noConstants['normalised'] = normalised
/var/folders/cm/m55y72dj6cd5lkwrfdb2bpnr0000gp/T/ipykernel_52880/1492737920.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adcdiff = [np.diff(signal) for signal in normalised]; noConstants['adcdiff'] = adcdiff
/var/folders/cm/m55y72dj6cd5lkwrfdb2bpnr0000gp/T/ipykernel_52880/1492737920.py:20: SettingWithCopyWarning: 
A value is trying to

In [26]:
# CUTS
onlyPositiveDiff = noConstants[noConstants['maxdiff'] >= 0].reset_index(drop=True); print(onlyPositiveDiff.shape) # Remove all clusters with only negative change in energy deposition
lessThan180 = onlyPositiveDiff[onlyPositiveDiff['height'] < 180]; print(lessThan180.shape) # Remove all clusters wit length in wire greater than 180 - muons, noise
ADC200cut = lessThan180[lessThan180['max_intensity'] >= 200]; print(ADC200cut.shape) # Remove all clusters with max ADC less than 200, noise

(105441, 30)
(91658, 30)
(33701, 30)


In [27]:
# Bounding Box in Each Plane Cuts

collection = (
    (onlyPositiveDiff['plane'] == 'collection') &
    (onlyPositiveDiff['bbox_min_row'] > 12) & (onlyPositiveDiff['bbox_min_row'] < 37) &
    (onlyPositiveDiff['bbox_max_col'] > 789) & (onlyPositiveDiff['bbox_max_col'] < 1927)
)

# Cuts for induction plane (adjust values as needed)
induction = (
    (onlyPositiveDiff['plane'] == 'induction') &
    (onlyPositiveDiff['bbox_min_row'] > 11) & (onlyPositiveDiff['bbox_min_row'] < 35) &
    (onlyPositiveDiff['bbox_max_col'] > 786) & (onlyPositiveDiff['bbox_max_col'] < 1794)
)

# Combine both cuts
bboxCut = onlyPositiveDiff[collection | induction].reset_index(drop=True); print(bboxCut.shape)

(31607, 30)
